In [2]:
!pip install gpt3_tokenizer

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls drive/MyDrive/llm-test-football-data/

Arsenal.txt	     Everton.txt	  Mallorca.txt	       Real_Madrid.txt
Aston_Villa.txt      FIFA.txt		  Manchester_City.txt  Real_Sociedad.txt
Athletic_Club.txt    Football.txt	  Man_City.txt	       Robert_Lewandowski.txt
Atlético_Madrid.txt  Fulham.txt		  Man_United.txt       Sevilla.txt
Barcelona.txt	     Getafe.txt		  Newcastle.txt        Tottenham.txt
Brentford.txt	     Girona.txt		  Nottm_Forest.txt     UEFA_Champions_League.txt
Brighton.txt	     Granada.txt	  Osasuna.txt	       UEFA.txt
Celta_Vigo.txt	     Jude_Bellingham.txt  Premiere_League.txt  Valencia.txt
Chelsea.txt	     La_Liga.txt	  Rayo_Vallecano.txt   Villarreal.txt
Crystal_Palace.txt   Liverpool.txt	  Real_Betis.txt       West_Ham.txt


In [5]:
import gpt3_tokenizer



tokenize = lambda s  : gpt3_tokenizer.encode(s)
revert = lambda t : gpt3_tokenizer.decode(t)


import os
# assign directory
directory = 'drive/MyDrive/llm-test-football-data/'

def get_training_data():
	print("[*]getting data")
	data = []
	for filename in os.listdir(directory):
		f = os.path.join(directory, filename)
   		# checking if it is a file
		if os.path.isfile(f):
			print(f)
			file = open(f, 'rt')
			data.append(tokenize(file.read()))
	return sum(data, [])


Model

In [6]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
training_data = get_training_data()

tokens = sorted(list(set(training_data)))
mapping = dict((c, i) for i, c in enumerate(tokens))

def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences
sequences = create_seq(training_data)

[*]getting data
drive/MyDrive/llm-test-football-data/Football.txt
drive/MyDrive/llm-test-football-data/La_Liga.txt
drive/MyDrive/llm-test-football-data/Real_Madrid.txt
drive/MyDrive/llm-test-football-data/Barcelona.txt
drive/MyDrive/llm-test-football-data/Athletic_Club.txt
drive/MyDrive/llm-test-football-data/Girona.txt
drive/MyDrive/llm-test-football-data/Atlético_Madrid.txt
drive/MyDrive/llm-test-football-data/Real_Sociedad.txt
drive/MyDrive/llm-test-football-data/Real_Betis.txt
drive/MyDrive/llm-test-football-data/Valencia.txt
drive/MyDrive/llm-test-football-data/Getafe.txt
drive/MyDrive/llm-test-football-data/Villarreal.txt
drive/MyDrive/llm-test-football-data/Osasuna.txt
drive/MyDrive/llm-test-football-data/Sevilla.txt
drive/MyDrive/llm-test-football-data/Celta_Vigo.txt
drive/MyDrive/llm-test-football-data/Mallorca.txt
drive/MyDrive/llm-test-football-data/Rayo_Vallecano.txt
drive/MyDrive/llm-test-football-data/Granada.txt
drive/MyDrive/llm-test-football-data/Premiere_League.txt
d

In [ ]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y )
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

# New Section